<a href="https://colab.research.google.com/github/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/blob/main/08_nlp_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py

--2025-11-02 08:55:40--  https://raw.githubusercontent.com/FairozAhmadSheikh/DeepLearning_Bootcamp_with_Tensor_Flow/refs/heads/main/helper_functions/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6733 (6.6K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   6.58K  --.-KB/s    in 0s      

2025-11-02 08:55:40 (71.9 MB/s) - ‘helper_functions.py’ saved [6733/6733]



In [2]:
from helper_functions import unzip_data,plot_loss_curves,compare_historys,create_tensorboard_callback

In [3]:
# Get the Dataset from kaggle NLP with Disaster Tweets
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2025-11-02 08:56:03--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.207, 172.217.203.207, 173.194.215.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2025-11-02 08:56:03 (42.8 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
unzip_data("nlp_getting_started.zip")

In [5]:
# Become One with Dataset (Visualize)
import pandas as pd
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df['text'][1]

'Forest fire near La Ronge Sask. Canada'

In [7]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [8]:
# Shuffle Dataset
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
# What does Test Dataset Look like
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each classes
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [11]:
# How many Total Samples
len(train_df),len(test_df)

(7613, 3263)

# **Visualize some random training examples**

In [12]:
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
    _,text,target=row
    print(f'Target : {target} \n ',"(real disaster)" if target>0 else "not real disaster")
    print(f'Text : \n {text}')
    print('---\n')

Target : 1 
  (real disaster)
Text : 
 8/6/2015@1:32 PM: HAZARD IN HIGH TRAFFIC AREA at 5000 DEANS BRIDGE RD http://t.co/itZzKWfhG5
---

Target : 1 
  (real disaster)
Text : 
 Look at the previous battles. Citizens were committing suicide so to not be under American control. The bomb was the only way. @NBCNews
---

Target : 1 
  (real disaster)
Text : 
 Since 1940 the year Angela Sanders was born roundhouse kick related deaths have increased 13000 percent.
---

Target : 1 
  (real disaster)
Text : 
 Bridge collapse not natural calamity but man-made: MPP lambasts Congress - KanglaOnline http://t.co/jp9XylA3C5  #Yugvani
---

Target : 0 
  not real disaster
Text : 
 Traumatised after seeing a baby literally fall out of that lady. She only went for a wee great catch! #oneborn
---



In [13]:
#Splitting data into Train and Validation
from sklearn.model_selection import train_test_split

train_sentances,val_sentances,train_labels,val_labels=train_test_split(
                                                      train_df_shuffled['text'].to_numpy(),
                                                      train_df_shuffled['target'].to_numpy(),
                                                      test_size=0.1,
                                                      random_state=42)

In [14]:
# Check lengths
len(train_sentances),len(val_sentances),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

# **Convert Text into Numbers**

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
text_vectorizer=TextVectorization(
                                  max_tokens=10000, # Set max_tokens to a value
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=None,
                                  pad_to_max_tokens=True,
                                   )

In [16]:
max_vocab_length=100   # Max number of words to have in vocabulary
max_length=15  # Max length our sequence will be
text_vectorizer=TextVectorization(max_tokens=max_vocab_length,
                                  standardize='lower_and_strip_punctuation',
                                  split='whitespace',
                                  ngrams=None,
                                  output_mode='int',
                                  output_sequence_length=max_vocab_length,
                                  pad_to_max_tokens=True)

In [17]:
# Fit the Text Vectorizer
text_vectorizer.adapt(train_sentances)

In [18]:
# Create a sample sentance and tokenize it
sample_sentance=random.choice(train_sentances)
print(f' The Original Sentance \n {sample_sentance} \n The Text Vectorized Version \n')
text_vectorizer([sample_sentance])

 The Original Sentance 
 BigRigRadio Live Accident Awareness 
 The Text Vectorized Version 



<tf.Tensor: shape=(1, 100), dtype=int64, numpy=
array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])>

In [19]:
# Get uniqe words in vocab
words_in_vocab=text_vectorizer.get_vocabulary()
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[-5:]
print(f'The Number of words in vocab are : {len(words_in_vocab)}')
print(f'The Five most common words in vocab are : {top_5_words}')
print(f'The Five least words in vocab are : {bottom_5_words}')

The Number of words in vocab are : 100
The Five most common words in vocab are : ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
The Five least words in vocab are : [np.str_('buildings'), np.str_('rt'), np.str_('first'), np.str_('cant'), np.str_('see')]


In [20]:
# Creating a embedding layer that turns text vectorized into the embeding vectors
from tensorflow.keras.layers import Embedding

embedding=Embedding(input_dim=max_vocab_length,
                    output_dim=128,
                    )

In [21]:
# Get Random Sentance from trainning set
random_sentance=random.choice(train_sentances)
print(f'Original Text \n {random_sentance}\n')
sample_embedding=embedding(text_vectorizer([random_sentance]))
sample_embedding

Original Text 
 Hellfire Gargoyle Hoof coil http://t.co/2ii3Brc7NX



<tf.Tensor: shape=(1, 100, 128), dtype=float32, numpy=
array([[[-0.01464714,  0.02408503, -0.03126063, ...,  0.03380401,
          0.00012226,  0.04394302],
        [-0.01464714,  0.02408503, -0.03126063, ...,  0.03380401,
          0.00012226,  0.04394302],
        [-0.01464714,  0.02408503, -0.03126063, ...,  0.03380401,
          0.00012226,  0.04394302],
        ...,
        [ 0.03529508, -0.03907074, -0.01696974, ...,  0.00207776,
         -0.03433559,  0.0196332 ],
        [ 0.03529508, -0.03907074, -0.01696974, ...,  0.00207776,
         -0.03433559,  0.0196332 ],
        [ 0.03529508, -0.03907074, -0.01696974, ...,  0.00207776,
         -0.03433559,  0.0196332 ]]], dtype=float32)>

# ***Lets Now Create our Baseline Model***
its common to use ML model first or non DL model and then switch to DL model

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0=Pipeline([
    ('tfid',TfidfVectorizer()),
    ('clf',MultinomialNB()),
])

model_0.fit(train_sentances,train_labels)
baseline_score=model_0.score(val_sentances,val_labels)

print(f'Our Baseline Model Acheives Accuracy of : {baseline_score*100:.2f}')

Our Baseline Model Acheives Accuracy of : 79.27


In [23]:
# Model Predictions
baseline_preds=model_0.predict(val_sentances)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

# **Creating an evaluation function**

In [24]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support


def calculate_results(y_true,y_pred):
  model_accuracy=accuracy_score(y_true=y_true,y_pred=y_pred)
  model_precission,model_recall,model_f1,_=precision_recall_fscore_support(y_true=y_true,y_pred=y_pred,average='weighted')

  model_results={
     "accuracy":model_accuracy,
     "precsion":model_precission,
     "recall":model_recall,
     'f1_score':model_f1
  }
  return model_results

In [25]:
# Get Baseline Results
baseline_results=calculate_results(y_true=val_labels,y_pred=baseline_preds)

baseline_results

{'accuracy': 0.7926509186351706,
 'precsion': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# **Model1 : Feed Forward Neural Network ( Dense Model)**

In [26]:
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers

In [27]:
SAVE_DIR="ModelLogs"

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x) # Add this layer to reduce dimensions
outputs=layers.Dense(1,activation='sigmoid')(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_Dense")
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,929 (50.50 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_1_history=model_1.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name='model_1_dense')])

Saving TensorBoard log files to: ModelLogs/model_1_dense/20251102-085606
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5754 - loss: 0.6777 - val_accuracy: 0.5472 - val_loss: 0.6755
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5937 - loss: 0.6657 - val_accuracy: 0.6129 - val_loss: 0.6546
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6210 - loss: 0.6459 - val_accuracy: 0.6181 - val_loss: 0.6390
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6479 - loss: 0.6207 - val_accuracy: 0.6759 - val_loss: 0.6171
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6693 - loss: 0.6136 - val_accuracy: 0.6785 - val_loss: 0.6092


In [29]:
model_1.evaluate(val_sentances,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6637 - loss: 0.6276


[0.6092172265052795, 0.6784777045249939]

In [30]:
model_1_predict_probs=model_1.predict(val_sentances)
model_1_predict_probs

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


array([[0.5150557 ],
       [0.63412595],
       [0.68855953],
       [0.32349774],
       [0.417361  ],
       [0.60851073],
       [0.68502504],
       [0.62696177],
       [0.64120555],
       [0.27114585],
       [0.45424625],
       [0.51126987],
       [0.550298  ],
       [0.4351494 ],
       [0.5943207 ],
       [0.47057122],
       [0.25848943],
       [0.35500622],
       [0.40117204],
       [0.48943302],
       [0.24179922],
       [0.12689953],
       [0.3757979 ],
       [0.43165144],
       [0.4664024 ],
       [0.58531594],
       [0.14009628],
       [0.51048535],
       [0.22947092],
       [0.27130133],
       [0.42962727],
       [0.4436235 ],
       [0.5037643 ],
       [0.43802512],
       [0.50320196],
       [0.15224107],
       [0.6188094 ],
       [0.2679542 ],
       [0.24843478],
       [0.646282  ],
       [0.34750432],
       [0.5636617 ],
       [0.3714288 ],
       [0.49174494],
       [0.33348122],
       [0.5615391 ],
       [0.4265124 ],
       [0.716

In [31]:
model_1_predict_probs.shape

(762, 1)

In [32]:
model_1_predict_probs[0]

array([0.5150557], dtype=float32)

In [33]:
# Make the preictions comaparable with the actaul labels
model_1_predict_probs=tf.squeeze(tf.round(model_1_predict_probs))
model_1_predict_probs[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [34]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

# ***Calculate Model Results***

In [35]:
model_1_results=calculate_results(y_true=val_labels,y_pred=model_1_predict_probs)
model_1_results

{'accuracy': 0.678477690288714,
 'precsion': 0.6791457862996518,
 'recall': 0.678477690288714,
 'f1_score': 0.6733883220410968}

In [36]:
import numpy as np
np.array(list(model_1_results.values()))>np.array(list(baseline_results.values()))

array([False, False, False, False])

# ***Lets Visualize the embedding that Model learned using Tensorflow Projector***

In [37]:
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(100,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [38]:
model_1.summary()

Model: "model_1_Dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,789 (151.52 KB)

 Trainable params: 12,929 (50.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,860 (101.02 KB)

In [39]:
embed_weights=model_1.get_layer('embedding').get_weights()[0]
embed_weights,print(embed_weights.shape)

(100, 128)


(array([[-0.00813341, -0.02413034, -0.0355041 , ...,  0.02485041,
         -0.05802625, -0.01729614],
        [ 0.14456637,  0.19873323,  0.12959915, ..., -0.12518397,
          0.16226445,  0.20449887],
        [-0.16442972, -0.15830314, -0.15810919, ...,  0.1750941 ,
         -0.09075067, -0.1265591 ],
        ...,
        [ 0.03462689,  0.02452631,  0.02255324, ..., -0.07589585,
          0.03163324,  0.01641665],
        [-0.43512076, -0.3734084 , -0.30190778, ...,  0.34440956,
         -0.25789693, -0.37366384],
        [-0.33282328, -0.32896253, -0.19649403, ...,  0.29822913,
         -0.20689334, -0.25165024]], dtype=float32),
 None)

Write the weights to disk. To use the Embedding Projector, you will upload two files in tab separated format: a file of vectors (containing the embedding), and a file of meta data (containing the words).  This code is from the tensorflow documentation Here[https://www.tensorflow.org/text/guide/word_embeddings]

In [40]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [41]:
# After this download step use projector tensorflow to visualzize it
"""
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass
"""


"\ntry:\n  from google.colab import files\n  files.download('vectors.tsv')\n  files.download('metadata.tsv')\nexcept Exception:\n  pass\n"

# ***MODEL2 =  LSTM [LONG SHORT TERM MEMORY]***

In [42]:
from tensorflow.keras import layers


In [43]:
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.LSTM(64,return_sequences=True)(x)
x=layers.LSTM(64)(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_2=tf.keras.Model(inputs,outputs)

In [44]:
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [45]:
model_2.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 100, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,457 (388.50 KB)

 Trainable params: 99,457 (388.50 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
model_2_history=model_2.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LSTM")])

Saving TensorBoard log files to: ModelLogs/model_2_LSTM/20251102-085617
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 32s 130ms/step - accuracy: 0.5675 - loss: 0.6833 - val_accuracy: 0.5433 - val_loss: 0.6911
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 31s 144ms/step - accuracy: 0.5666 - loss: 0.6851 - val_accuracy: 0.5433 - val_loss: 0.6906
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 28s 129ms/step - accuracy: 0.5772 - loss: 0.6820 - val_accuracy: 0.5433 - val_loss: 0.6908
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 28s 131ms/step - accuracy: 0.5765 - loss: 0.6820 - val_accuracy: 0.5433 - val_loss: 0.6910
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 28s 131ms/step - accuracy: 0.5745 - loss: 0.6827 - val_accuracy: 0.5433 - val_loss: 0.6905


In [47]:
model_2_pred_probs=model_2.predict(val_sentances)
model_2_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 48ms/step


array([[0.43289074],
       [0.43289074],
       [0.43289074],
       [0.43289074],
       [0.43289074],
       [0.43289074],
       [0.43289074],
       [0.43289074],
       [0.43289074],
       [0.43289074]], dtype=float32)

In [48]:
model_2_preds=tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [49]:
model_2_results=calculate_results(y_true=val_labels,y_pred=model_2_preds)
model_2_results

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.5433070866141733,
 'precsion': 0.29518259036518074,
 'recall': 0.5433070866141733,
 'f1_score': 0.38253254057528524}

In [50]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precsion': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# ***As it is clear our baseline is more accurate***

# ***Model_3 GRU (Gated Recurrent Unit)***

In [51]:
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GRU(64)(x)
x=layers.Dense(64,activation='relu')(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_3=tf.keras.Model(inputs,outputs)


In [52]:
model_3.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,273 (212.00 KB)

 Trainable params: 54,273 (212.00 KB)

 Non-trainable params: 0 (0.00 B)

In [53]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [54]:
model_3_history=model_3.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_GRU")])

Saving TensorBoard log files to: ModelLogs/model_GRU/20251102-085846
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - accuracy: 0.5700 - loss: 0.6837 - val_accuracy: 0.5433 - val_loss: 0.6908
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 84ms/step - accuracy: 0.5694 - loss: 0.6843 - val_accuracy: 0.5433 - val_loss: 0.6900
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 83ms/step - accuracy: 0.5729 - loss: 0.6834 - val_accuracy: 0.5433 - val_loss: 0.6894
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 20s 93ms/step - accuracy: 0.5747 - loss: 0.6830 - val_accuracy: 0.5433 - val_loss: 0.6940
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 85ms/step - accuracy: 0.5727 - loss: 0.6832 - val_accuracy: 0.5433 - val_loss: 0.6939


In [55]:
model_3_pred_probs=model_3.predict(val_sentances)
model_3_pred_probs

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step


array([[0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.40980977],
       [0.409

In [56]:
model_3_pred=tf.squeeze(tf.round(model_3_pred_probs))
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [57]:
model_3_results=calculate_results(val_labels,model_3_pred)
model_3_results

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'accuracy': 0.5433070866141733,
 'precsion': 0.29518259036518074,
 'recall': 0.5433070866141733,
 'f1_score': 0.38253254057528524}

# ***GRU Does not beat the baseline***
lets try something else

# ***Model_4 Bidirectional Model***

In [58]:
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_4=tf.keras.Model(inputs,outputs)

model_4.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

model_4.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 128)       │        12,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 111,745 (436.50 KB)

 Trainable params: 111,745 (436.50 KB)

 Non-trainable params: 0 (0.00 B)

In [59]:
model_4_history=model_4.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bi_directional")])

Saving TensorBoard log files to: ModelLogs/model_4_bi_directional/20251102-090028
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 34s 141ms/step - accuracy: 0.7075 - loss: 0.5887 - val_accuracy: 0.6903 - val_loss: 0.6017
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 29s 133ms/step - accuracy: 0.7169 - loss: 0.5640 - val_accuracy: 0.6864 - val_loss: 0.5890
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 28s 130ms/step - accuracy: 0.7183 - loss: 0.5601 - val_accuracy: 0.6890 - val_loss: 0.5898
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 29s 134ms/step - accuracy: 0.7228 - loss: 0.5526 - val_accuracy: 0.6903 - val_loss: 0.5817
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 28s 129ms/step - accuracy: 0.7124 - loss: 0.5561 - val_accuracy: 0.6850 - val_loss: 0.5806


In [60]:
model_4_pred_probs=model_4.predict(val_sentances)
model_4_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step


array([[0.6153907 ],
       [0.80354655],
       [0.7891567 ],
       [0.24422196],
       [0.32511184],
       [0.84203154],
       [0.7724799 ],
       [0.7522647 ],
       [0.8812174 ],
       [0.16686273]], dtype=float32)

In [61]:
model_4_pred=tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [62]:
model_4_results=calculate_results(val_labels,model_4_pred)

In [63]:
model_4_results

{'accuracy': 0.6850393700787402,
 'precsion': 0.6840990371876214,
 'recall': 0.6850393700787402,
 'f1_score': 0.6840943475344582}

# ***Still Not enough to beat the baseline***

**1D Convolutional Neural Network**

In [64]:
from tensorflow.keras import layers

inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Conv1D(filters=64,kernel_size=5,strides=1,activation='relu',padding='valid')(x)
x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation='sigmoid')(x)
model_5=tf.keras.Model(inputs,outputs,name='Model_5_Conv1D')

In [65]:
model_5.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [66]:
model_5_history=model_5.fit(train_sentances,train_labels,epochs=5,validation_data=(val_sentances,val_labels))

Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - accuracy: 0.7038 - loss: 0.5809 - val_accuracy: 0.6850 - val_loss: 0.6092
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.7136 - loss: 0.5718 - val_accuracy: 0.6969 - val_loss: 0.6280
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.7369 - loss: 0.5442 - val_accuracy: 0.6969 - val_loss: 0.5755
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - accuracy: 0.7396 - loss: 0.5406 - val_accuracy: 0.7073 - val_loss: 0.5879
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.7362 - loss: 0.5389 - val_accuracy: 0.7034 - val_loss: 0.6015


In [67]:
model_5_pred_probs=model_5.predict(val_sentances)
model_5_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


array([[0.49881083],
       [0.8443475 ],
       [0.7535941 ],
       [0.25840428],
       [0.3178914 ],
       [0.9215903 ],
       [0.9954459 ],
       [0.89370924],
       [0.9343717 ],
       [0.07098875]], dtype=float32)

In [68]:
model_5_pred=tf.squeeze(tf.round(model_5_pred_probs))
model_5_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [69]:
model_5_results=calculate_results(val_labels,model_5_pred)
model_5_results

{'accuracy': 0.7034120734908137,
 'precsion': 0.7037099026639765,
 'recall': 0.7034120734908137,
 'f1_score': 0.7002008518698009}

# **Still does not beat the Baseline**
 Lets try a pretrained  model from tensorflow hub
 Using *USE: Unviversal Sentence Encoder*

In [70]:
import tensorflow_hub as hub
embed=hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')

In [71]:
model_url='https://tfhub.dev/google/universal-sentence-encoder/4'

In [72]:
embed_samples=embed([sample_sentance])
print(embed_samples[0][:50])

tf.Tensor(
[ 0.05550208  0.04709413 -0.01895739 -0.07902899  0.03958949  0.00876916
 -0.04969131  0.04989154 -0.00747396 -0.00288077  0.06884657  0.04953969
 -0.00839321 -0.06465396  0.02695627 -0.06751097 -0.03387656 -0.02691915
  0.04167464 -0.07064907 -0.06623215  0.03144345  0.07820048 -0.02913984
 -0.05731257  0.02590276  0.05266502  0.01158521 -0.06334284  0.00348307
 -0.00446465  0.02877871  0.064885   -0.03860526 -0.06959674  0.04759655
 -0.09089859 -0.02090189  0.0557386  -0.04193855 -0.0489285  -0.00412812
  0.07782833 -0.06199645 -0.05861899 -0.00132393 -0.0067562   0.0176226
 -0.06589837  0.02317566], shape=(50,), dtype=float32)


In [73]:
sentence_encoder_layer=hub.KerasLayer(model_url,input_shape=[],dtype=tf.string,trainable=False,
                                      name='Universal_sentence_enocoder')

In [74]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, Model
import numpy as np


# 1. Fix for TensorFlow-Hub + Keras 3 incompatibility

class USEEncoder(layers.Layer):
    def __init__(self, model_url, trainable=False, **kwargs):
        super().__init__(**kwargs)
        self.use_layer = hub.load(model_url)
        self.trainable = trainable

    def call(self, inputs):
        return self.use_layer(inputs)





In [75]:
# 2. Load the Universal Sentence Encoder

print("Loading Universal Sentence Encoder (this may take a few seconds)...")
model_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
use_layer = USEEncoder(model_url)




Loading Universal Sentence Encoder (this may take a few seconds)...


In [91]:
# Build Model
inputs = layers.Input(shape=(), dtype=tf.string, name="input_text")
embeddings = use_layer(inputs)
x = layers.Dense(256, activation="relu")(embeddings)
x = layers.Dropout(0.3)(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_6= Model(inputs=inputs, outputs=outputs, name="USE_Sentiment_Model")

In [92]:
# compile
model_6.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [93]:
# Train
print("\nTraining model...\n")
model_6_history=model_6.fit(train_sentances, train_labels, epochs=5, batch_size=2, verbose=1)


Training model...

Epoch 1/5
3426/3426 ━━━━━━━━━━━━━━━━━━━━ 35s 9ms/step - accuracy: 0.7747 - loss: 0.4851
Epoch 2/5
3426/3426 ━━━━━━━━━━━━━━━━━━━━ 33s 10ms/step - accuracy: 0.8388 - loss: 0.3839
Epoch 3/5
3426/3426 ━━━━━━━━━━━━━━━━━━━━ 31s 9ms/step - accuracy: 0.8553 - loss: 0.3485
Epoch 4/5
3426/3426 ━━━━━━━━━━━━━━━━━━━━ 31s 9ms/step - accuracy: 0.8805 - loss: 0.2919
Epoch 5/5
3426/3426 ━━━━━━━━━━━━━━━━━━━━ 30s 9ms/step - accuracy: 0.9040 - loss: 0.2346


In [94]:
#  Predict
print("\nEvaluating on validation sentences...\n")
model_6_predictions = model_6.predict(val_sentances)

for sent, pred in zip(val_sentances, model_6_predictions):
    label = "Positive 😊" if pred > 0.5 else "Negative 😞"
    print(f"Sentence: {sent}\nPrediction: {label} (score: {pred[0]:.4f})\n")


Evaluating on validation sentences...

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
Sentence: DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5
Prediction: Negative 😞 (score: 0.0674)

Sentence: FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday
Prediction: Positive 😊 (score: 0.6375)

Sentence: Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj
Prediction: Positive 😊 (score: 0.9999)

Sentence: @camilacabello97 Internally and externally screaming
Prediction: Negative 😞 (score: 0.1320)

Sentence: Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov
Prediction: Positive 😊 (score: 0.6208)

Sentence: Investigators rule catastrophic structural failure resulted in 2014 Virg.. Related Articles: http:

In [80]:
model_6_preds=tf.squeeze(tf.round(model_6_predictions))

In [81]:
model_6_results=calculate_results(val_labels,model_6_preds)

In [82]:
model_6_results

{'accuracy': 0.8070866141732284,
 'precsion': 0.8108989089317052,
 'recall': 0.8070866141732284,
 'f1_score': 0.8049387920419209}

In [86]:
#  Save model

model.save("use_sentiment_model_fixed.h5")
print("✅ Model saved as 'use_sentiment_model_fixed.h5'")

✅ Model saved as 'use_sentiment_model_fixed.h5'


# **Model 7 :**
**TF Pretrained USE With 10% traiining data**


In [101]:
train_10_percent = train_df_shuffled[['text', 'target']].sample(frac=0.1, random_state=42)
train_10_percent.head()

,text,target
4955,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0
584,FedEx no longer to transport bioterror germs i...,0
7411,Gunmen kill four in El Salvador bus attack: Su...,1
5950,@camilacabello97 Internally and externally scr...,1
5541,Radiation emergency #preparedness starts with ...,1


In [102]:
train_sentences_10_percent = train_10_percent['text'].astype(str).tolist()
train_labels_10_percent = train_10_percent['target'].astype(int).tolist()


In [103]:
print("10% sample distribution:\n", train_10_percent['target'].value_counts())
print("\nFull dataset distribution:\n", train_df_shuffled['target'].value_counts())

10% sample distribution:
 target
0    413
1    348
Name: count, dtype: int64

Full dataset distribution:
 target
0    4342
1    3271
Name: count, dtype: int64


In [105]:
model_7 = tf.keras.models.clone_model(model_6)
model_7.set_weights(model_6.get_weights())

In [106]:
model_7.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [107]:
model_7.summary()

Model: "USE_Sentiment_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_text (InputLayer)         │ (None)                 │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ use_encoder (USEEncoder)        │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147,841 (577.50 KB)

 Trainable params: 147,841 (577.50 KB)

 Non-trainable params: 0 (0.00 B)

In [108]:
model_7_history = model_7.fit(
    x=np.array(train_sentences_10_percent, dtype=object),  # important: array of dtype=object for text
    y=np.array(train_labels_10_percent),
    epochs=5,
    validation_data=(np.array(val_sentances, dtype=object), np.array(val_labels))
)

Epoch 1/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 6s 61ms/step - accuracy: 0.8263 - loss: 0.5078 - val_accuracy: 0.8438 - val_loss: 0.3934
Epoch 2/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.8442 - loss: 0.3991 - val_accuracy: 0.8675 - val_loss: 0.3434
Epoch 3/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.8484 - loss: 0.3647 - val_accuracy: 0.8819 - val_loss: 0.3028
Epoch 4/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.8786 - loss: 0.3019 - val_accuracy: 0.8911 - val_loss: 0.2683
Epoch 5/5
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.8804 - loss: 0.2901 - val_accuracy: 0.9003 - val_loss: 0.2346


In [109]:
model_7_pred_probs=model_7.predict(val_sentances)
model_7_pred_probs[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step


array([[0.04944657],
       [0.5867713 ],
       [0.99994135],
       [0.36973447],
       [0.9110848 ],
       [0.992219  ],
       [0.99491006],
       [0.99997264],
       [0.9991953 ],
       [0.05820926]], dtype=float32)

In [110]:
model_7_preds=tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [111]:
# Evaluate
model_7_results=calculate_results(val_labels,model_7_preds)

In [112]:
model_7_results

{'accuracy': 0.9002624671916011,
 'precsion': 0.9080656451024357,
 'recall': 0.9002624671916011,
 'f1_score': 0.8990580575052554}